# Objetivo

Prever a avaliação média de um filme baseado na popularidade dos 10 filmes mais próximos (em termos de gênero e popularidade).

## Preparando os datasets

### Data Frame de avaliações

In [1]:
import pandas as pd
r_cols = ['user_id', 'movie_id', 'rating'] #colunas que serão usadas no Data Frame
ratings = pd.read_csv('../../dependencias/arquivos_de_codigo/u.data', sep='\t', names=r_cols, usecols=range(3))
#define um DF a partir do csv u.data, tomando o separador com um tab e as colunas como sendo as 3 primeiras da tabela csv
ratings.head()

,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


Como estamos interessados nas avaliações dos filmes, vamos agregar as informações da coluna 'rating' e agrupar por filme (movie_id):

In [2]:
import numpy as np

movieProperties = ratings.groupby('movie_id').agg({'rating':[np.size, 'mean']})
#usa multiindex para criar "subcolunas" com o total e a media das avaliações por movie_id, informações que serão usadas depois
movieProperties.head()

rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

O valor absoluto de avaliações (linhas da coluna "size") não são tão úteis para medir popularidade, já que esta é uma qualidade de proporção. Portanto, é necessário normalizar esses valores.

In [3]:
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
#separa o número total de avaliações de cada filme (cada movie_id) num DF
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x))/(np.max(x) - np.min(x)))
#aplica a função lambda na coluna de 'size' de movieNumRatings
#normaliza a partir da amplitude do dataset (filme mais avaliado - menos avaliado)
#gera floats de 0 a 1, quanto mais próximo de 0, menos popular, e quanto mais próximo de 1, mais popular
movieNormalizedNumRatings.head()

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


Para se estimar a avaliação do filme, será necessário comparar seus gêneros e popularidade com todos os outros para calcular a distância. Para isso, será útil a criação de um dicionário contendo essas informações filme por filme, o que é feito no bloco de código abaixo:

In [4]:
movieDict = {}
with open(r'../../dependencias/arquivos_de_codigo/u.item', encoding='ISO-8859-1') as f:
    for line in f:
        fields = line.rstrip('\n').split('|')
        #separa os campos de cada registro por '|' e remove o trailing character de nova linha
        movieID = int(fields[0])
        #a ID do filme é a primeira secção do registro
        name = fields[1]
        #o nome é a segunda
        genres = fields[5:25]
        #os há 19 campos de 0s e 1s que indicam se um filme faz parte ou não do gênero referente aquele campo
        genres = map(int, genres) #converte os 0s e 1s, que vem em formato de string, para inteiro
        #movieDict[movieID] = (name, np.array(list(genres)), movieNormalizedNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))
        movieDict[movieID] = (name, np.array(list(genres)), movieNormalizedNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))
        #cria uma tupla como item de movieDict, com o nome, gêneros, popularidade, e a nota média das avaliações

Agora que temos as informações sobre filme, seus generos e sua popularidade (número de avaliações normalizado) facilmente acessíveis e relacionáveis em um dicionário, podemos escrever uma função para calcular a distância entre dois filmes (itens de movieDict) baseando-se em seus gêneros e popularidade:

In [5]:
from scipy import spatial #pacote com funções para calculos espaciais com diferentes estruturas dados

def ComputeDistance(a, b): #a e b são itens do dicionario movieDict
    genresA = a[1] #campo 1 da tupla é o array com as flags de gênero do filme
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB)
    #calcula o cosseno entre os vetores genresA e genresB. Como possuem apenas valores não negativos, o resultado é entre 0 e 1
    #quanto mais proximo de 0, mais proximos de paralelos são os vetores, e quanto mais proximo de 1, mais proximos de ortogonais
    #portanto, quanto menor, mais parecidos e menor a distancia entre os generos dos filmes
    popularityA = a[2] #campo 2 da tupla é o da popularidade (número de avaliações normalizado)
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB) #a distancia entre as popularidades será apenas a diferença algébrica entre elas
    return genreDistance + popularityDistance #a distancia total entre dois filmes será a soma das distancias de genero e popularidade
    #quanto maior essa soma, mais distantes são os filmes

ComputeDistance(movieDict[2], movieDict[4]) #testando a função 

0.8004574042309892

In [6]:
print(movieDict[2])
print(movieDict[4])

('GoldenEye (1995)', array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]), 0.22298456260720412, 3.2061068702290076)
('Get Shorty (1995)', array([0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.3567753001715266, 3.550239234449761)


A função ComputeDistance retornou um valor que indica que os filmes são consideravelmente diferentes.
E, de fato, um é de ação, e o outro é de comédia. Além disso, o filme 4 é aproximadametne 13 pp mais popular.

Agora, podendo calcular a distância entre os filmes, podemos capturar os 10 filmes mais próximos de um filme teste, e prever sua nota média calculando a média das avaliações desses 10 filmes.

In [7]:
import operator #biblioteca com a função itemgetter, que será usada para acessar uma informação numa tupla

def getNeighbors(movieID, K): #retorna uma lista com as IDs dos K filmes mais próximos do filme de ID movieID
    distances = []
    for movie in movieDict: #itera por todos os itens de movieDict para calcular a distancia do filme com todos os outros
        if (movie != movieID): #verificar se o filme da iteração não é o de referência, comparando as IDs (que são as chaves do dict)
            dist = ComputeDistance(movieDict[movieID], movieDict[movie]) #calcula a distancia entre os filmes
            distances.append((movie, dist)) #adiciona a distancia e o ID do filme para o qual é essa distancia, em uma tupla
    distances.sort(key=operator.itemgetter(1))
    #parametro key recebe uma função a ser aplicada em cada elemento da lista e o valor retornado ser a referencia para a ordenação
    #a chamada itemgetter(x) gera uma função que sempre pega o elemento de index x do objeto passado como argumento
    #no fim, essa linha ordena a lista de distancias crescentemente pelo parâmetro de distancia de cada tupla
    neighbors = []
    for x in range(K): #adiciona à lista neighbors a ID dos K filmes de menor distancia
        neighbors.append(distances[x][0])
    return neighbors #retorna a lista com os K vizinhos mais proximos do filme de ID movieID

#testando usando o filme Toy Story de referência:
K = 10
avgRating = 0
neighbors = getNeighbors(1, K)
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3] / K #calcula a media das notas dos vizinhos, que será a previsao para a media de Toy Story
    print(f'{movieDict[neighbor][0]} {movieDict[neighbor][3]}') #imprime o nome do vizinho com sua nota média

Liar Liar (1997) 3.156701030927835
Aladdin (1992) 3.8127853881278537
Willy Wonka and the Chocolate Factory (1971) 3.6319018404907975
Monty Python and the Holy Grail (1974) 4.0664556962025316
Full Monty, The (1997) 3.926984126984127
George of the Jungle (1997) 2.685185185185185
Beavis and Butt-head Do America (1996) 2.7884615384615383
Birdcage, The (1996) 3.4436860068259385
Home Alone (1990) 3.0875912408759123
Aladdin and the King of Thieves (1996) 2.8461538461538463


Toy Story é um filme infantil, de comédia e popular, o que também pode ser dito desses 10 filmes.

In [8]:
print(f'Predição da nota média de Toy Story: {avgRating}\nA nota, de fato: {movieDict[1][3]}')
print(f'Erro relativo de: {100 * (abs(avgRating - movieDict[1][3])/movieDict[1][3])}%')

Predição da nota média de Toy Story: 3.344590590023556
A nota, de fato: 3.8783185840707963
Erro relativo de: 13.761839892147897%


Como podemos ver, a predição foi próxima do valor real, com um erro pouco maior que 10%.

## Activity

Our choice of 10 for K was arbitrary - what effect do different K values have on the results?

Our distance metric was also somewhat arbitrary - we just took the cosine distance between the genres and added it to the difference between the normalized popularity scores. Can you improve on that?